# Schema-First Vertex-Model Simulations with Tyssue and process-bigraph

This notebook demonstrates a process-bigraph wrapper for the popular cell-vertex model simulator for epithelial tissue: Tyssue

In [ ]:
import random
import time
import math
import pandas as pd
from pprint import pprint

from bigraph_schema import allocate_core
from process_bigraph import Composite
from process_bigraph.emitter import emitter_from_wires, gather_emitter_results
from bigraph_viz import plot_bigraph

from vivarium_tyssue.maps import *
from vivarium_tyssue.core_maps import GEOMETRY_MAP
from vivarium_tyssue.processes.eulersolver import (
    EulerSolver,
    get_test_spec,
    run_test_solver,
    get_test_config_flat,
    get_test_config,
)
from vivarium_tyssue.processes.regulations import (
    TestRegulations,
    StochasticLineTension,
    CellJamming,
    get_test_jamming_spec,
    get_test_regulation_spec,
    get_test_stochastic_spec,

)
from vivarium_tyssue.data_types import register_types
from vivarium_tyssue.processes import register_processes

from matplotlib import pyplot as plt
from tyssue import config
from tyssue.draw import create_gif
from IPython.display import Image, display
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
import matplotlib.colors as colors
from matplotlib import cm

from bigraph_viz import plot_bigraph

#### Loading all relevant processes

In [ ]:
# create the core object
core = allocate_core()
# register processes
core = register_types(core)
core.register_link("EulerSolver", EulerSolver)
core.register_link("TestRegulations", TestRegulations)
core.register_link("StochasticLineTension", StochasticLineTension)
core.register_link("CellJamming", CellJamming)
core = register_processes(core)

## Basic Simulation

Simulating a rectangular epithelial tissue with a basic vertex-model formulation as found in [Bi et al., 2015]. The total potential energy of the tissue is determined by the following energy function:


<center>
$E = \sum_{i=1}^{N}[K_{A_{i}}(A_{i}-A_{i0})^{2} + K_{P_{i}}(P_{i}-P_{i0})^{2}]$
</center>

<p>
Here, the sum is over each cell $i$. $K_{A_{i}}$ is the area elasticity, $A_{i}$ and $A_{i0}$ are the current area and preferred area of cell $i$, $K_{P_{i}}$ is the area elasticity, and $P_{i}$ and $P_{i0}$ are the current area and preferred area of cell $i$.
</p>

In [ ]:
config1 = {
        "name": "Test Square",
        "eptm": "test_square.hf5",
        "tissue_type": "Sheet",
        "parameters": {
            "face_df": {
                "area_elasticity": 1,
                "prefered_area": 1,
                "perimeter_elasticity": 0.1,
                "prefered_perimeter": 3.6,
                "is_alive": 1,
            },
            "edge_df": {
                "line_tension": 0,
                "is_active": 1,
            },
            "vert_df": {
                "viscosity": 1,
                "is_alive": 1,
            }
        },
        "geom": "SheetGeometry",
        "effectors": ["LineTension", "FaceAreaElasticity", "PerimeterElasticity"],
        "ref_effector": "FaceAreaElasticity",
        "factory": "model_factory_bound",
        "settings": {
            "threshold_length": 0.03
        },
        "auto_reconnect": True, # if True, will automatically perform reconnections
        "bounds": None, # bounds the displacement of the vertices at each time step
        "output_columns": {} # dict containing lists of column names to emit for each dataframe
    }

In [ ]:
spec1 = {
        "Tyssue": {
            "_type": "process",
            "address": "local:EulerSolver",
            "config": config1,
            "inputs": {
                "behaviors": ["Behaviors"],
                "global_time": ["global_time"],
            },
            "outputs": {
                "datasets": ["Datasets"],
                "network_changed": ["Network Changed"],
                "behaviors_update": ["Behaviors"],
            },
            "interval": 0.1,
        },
        "Network Changed": False,
        "Behaviors": {}
    }

In [ ]:
plot_bigraph(spec1, core=core)

### Run Simulation

In [ ]:
spec1["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        "face_df": ["Datasets", "face_df"],
        "edge_df": ["Datasets", "edge_df"],
        "vert_df": ["Datasets", "vert_df"],
    })
sim1 = Composite(
    {
        "state": spec1,
    },
    core=core,
)
sim1.run(20)
results = gather_emitter_results(sim1)[("emitter",)]
#TODO: Add initial tissue figure

### Generate Movie

In [ ]:
history = sim1.state["Tyssue"]["instance"].history
history.update_datasets()
draw_specs = config.draw.sheet_spec()
cmap = plt.get_cmap("autumn")
color_map = cmap([0.0 if i == 33 else 0.0 for i in range(206)])
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.5
draw_specs["face"]["color"] = color_map
draw_specs["edge"]["color"] = "black"
create_gif(history, "test_flat.gif", coords=["x", "y"], num_frames = 200, **draw_specs)

In [ ]:
display(Image(filename="test_flat.gif"))

## Cell Division

Next we simulate ***cell divisions***. Normally, cell divisions would have to be pre-defined during simulation setup. With our Compositional approach, external processes can trigger divisions - potentially dynamic processes. Here we simulate divisions occurring periodically.

In [ ]:
spec2 = get_test_regulation_spec(interval=0.1, config=config1)


In [ ]:
plot_bigraph(spec2, core=core)

In [ ]:
spec2["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        "face_df": ["Datasets", "face_df"],
        "edge_df": ["Datasets", "edge_df"],
        "vert_df": ["Datasets", "vert_df"],
    })
sim2 = Composite(
    {
        "state": spec2,
    },
    core=core,
)
sim2.run(20)
results2 = gather_emitter_results(sim2)[("emitter",)]

In [ ]:
history = sim2.state["Tyssue"]["instance"].history
history.update_datasets()
draw_specs = config.draw.sheet_spec()
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.5
draw_specs["face"]["color"] = "crimson"
draw_specs["edge"]["color"] = "black"
create_gif(history, "test_flat_division.gif", coords=["x", "y"], num_frames = 200, **draw_specs)

In [ ]:
display(Image(filename="test_flat_division.gif"))

## Stochastic Junctional Tension

Next, we simulate auto-correlated stochastic fluctuations in junctional tension between cells. Such fluctuations occur due to stochastic changes in the Actomyosin cortex of cells. The energy function is altered as follows:

<center>$E = \sum_{i=1}^{N}[K_{A_{i}}(A_{i}-A_{i0})^{2} + K_{P_{i}}(P_{i}-P_{i0})^{2}] + \sum_{\lt i,j\gt }^{}\Delta\Lambda_{ij}(t)l_{ij}$</center>

Here, the additional term is a line-tension with magnitude $\Delta\Lambda_{ij}(t)$, while $l_{ij}$ is the length of the junction between cell *i* and *j*.

The time-correlated tension fluctuations are as follows:

<center>$\frac{d\Delta\Lambda(t)}{dt} = -\frac{\Delta\Lambda(t)_{ij}}{\tau} + \xi_{ij}(t)$</center>

Where, $\left\langle \xi_{ij}(t) \right\rangle = 0$, $\left\langle \xi_{ij}(t_{1})\xi_{kl}(t_{2}) \right\rangle = \delta_{ik}\delta_{jl}\delta(t1-t2)$,  $\left\langle \Delta\Lambda_{ij}(t) \right\rangle = 0$, and $\left\langle \Delta\Lambda_{ij}(t_{1})\Delta\Lambda_{kl}(t_{2}) \right\rangle = \delta_{ik}\delta_{jl}\sigma^{2}e^{-\frac{\left| t_{1} - t_{2} \right|}{\tau}}$
The resulting process has a time-correlated and white-noise term.

These calculations are performed by a `StochasticTension` process.


In [ ]:
spec3 = get_test_stochastic_spec(interval=0.1, config=config1, tau=0.1, sigma=0.3)

In [ ]:
plot_bigraph(spec3, core=core)

In [ ]:
spec3["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        "face_df": ["Datasets", "face_df"],
        "edge_df": ["Datasets", "edge_df"],
        "vert_df": ["Datasets", "vert_df"],
    })
sim3 = Composite(
    {
        "state": spec3,
    },
    core=core,
)
sim3.run(20)
results3 = gather_emitter_results(sim3)[("emitter",)]

In [ ]:
history = sim3.state["Tyssue"]["instance"].history
history.update_datasets()
draw_specs = config.draw.sheet_spec()
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.5
draw_specs["face"]["color"] = "crimson"
draw_specs["edge"]["color"] = "black"
create_gif(history, "test_flat_stochastic.gif", coords=["x", "y"], num_frames = 200, **draw_specs)

In [ ]:
display(Image(filename="test_flat_stochastic.gif"))

## Cell Migration

We simulate a single migrating cell. No additional processes are required for this, just an additional force.

In [ ]:
spec4 = get_test_stochastic_spec(interval=0.01, config=get_test_config_flat(), tau=0.2, sigma=0.1)
plot_bigraph(spec4, core=core)

In [ ]:
pprint(spec4)

In [ ]:
spec4["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        "face_df": ["Datasets", "face_df"],
        "edge_df": ["Datasets", "edge_df"],
        "vert_df": ["Datasets", "vert_df"],
    })
sim4 = Composite(
    {
        "state": spec4,
    },
    core=core,
)
sim4.run(200)
results4 = gather_emitter_results(sim4)[("emitter",)]

In [ ]:
history = sim4.state["Tyssue"]["instance"].history
history.update_datasets()
draw_specs = config.draw.sheet_spec()
cmap = plt.get_cmap("autumn")
color_map = cmap([1 if i == 33 else 0.0 for i in range(206)])
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.5
draw_specs["face"]["color"] = color_map
draw_specs["edge"]["color"] = "black"
draw_specs["edge"]["color"] = "black"
create_gif(history, output="test_stochastic_migration.gif", coords = ["x", "y"], **draw_specs, num_frames=200)

In [ ]:
display(Image(filename="test_stochastic_migration.gif"))

## Cell Jamming

Cell jamming is a process by which tissues become more solid-like and stop collective migration. Cell become less elongated and more round. This is caused by a decrease in the preferred preimeter of the cells.

In [ ]:
spec5 = get_test_jamming_spec(interval=0.01, config=get_test_config_flat(), tau=0.2, sigma=0.1)
plot_bigraph(spec5, core=core)

In [ ]:
spec5["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        "face_df": ["Datasets", "face_df"],
        "edge_df": ["Datasets", "edge_df"],
        "vert_df": ["Datasets", "vert_df"],
    })
sim5 = Composite(
    {
        "state": spec5,
    },
    core=core,
)
sim5.run(200)
results5 = gather_emitter_results(sim4)[("emitter",)]

In [ ]:
history = sim5.state["Tyssue"]["instance"].history
history.update_datasets()
draw_specs = config.draw.sheet_spec()
cmap = plt.get_cmap("autumn")
color_map = cmap([1 if i == 33 else 0.0 for i in range(206)])
draw_specs["face"]["visible"] = True
draw_specs["face"]["alpha"] = 0.5
draw_specs["face"]["color"] = color_map
draw_specs["edge"]["color"] = "black"
draw_specs["edge"]["color"] = "black"
create_gif(history, output="test_jamming_flat.gif", coords = ["x", "y"], **draw_specs, num_frames=200)

In [ ]:
display(Image(filename="test_jamming_flat.gif"))
#TODO: fix boundaries, and find experimental videos

In [ ]:
history

In [ ]:
data = {
    "x": [],
    "y": [],
    "time": [],
}
for key, value in history.dicts["face"].items():
    data["time"].append(key)
    data["x"].append(value.loc[value["face"]==33, "x"].values[0])
    data["y"].append(value.loc[value["face"]==33, "y"].values[0])

df = pd.DataFrame(data)
df["time"] = df["time"].astype(float)
dx = df["x"].diff()
dy = df["y"].diff()

df["step_disp"] = np.sqrt(dx**2 + dy**2)
df["cum_disp"] = df["step_disp"].cumsum()
df["cum_disp"] = df["cum_disp"].fillna(0)

In [ ]:
history2 = sim4.state["Tyssue"]["instance"].history
data = {
    "x": [],
    "y": [],
    "time": [],
}
for key, value in history2.dicts["face"].items():
    data["time"].append(key)
    data["x"].append(value.loc[value["face"]==33, "x"].values[0])
    data["y"].append(value.loc[value["face"]==33, "y"].values[0])

df2 = pd.DataFrame(data)
df2["time"] = df2["time"].astype(float)
dx = df2["x"].diff()
dy = df2["y"].diff()

df2["step_disp"] = np.sqrt(dx**2 + dy**2)
df2["cum_disp"] = df2["step_disp"].cumsum()
df2["cum_disp"] = df2["cum_disp"].fillna(0)

In [ ]:
df["time"] = pd.to_numeric(df["time"])
df2["time"] = pd.to_numeric(df2["time"])

t_min = min(df["time"].min(), df2["time"].min())
t_max = max(df["time"].max(), df2["time"].max())

norm = colors.Normalize(vmin=t_min, vmax=t_max)
cmap = plt.cm.autumn
cmap2 = plt.cm.viridis

In [ ]:
def colored_trajectory(df_in, ax, label, cmap, norm):
    points = np.column_stack([df_in["x"], df_in["y"]])
    segments = np.stack([points[:-1], points[1:]], axis=1)

    lc = LineCollection(
        segments,
        cmap=cmap,
        norm=norm,
        linewidth=2,
    )
    lc.set_array(df_in["time"].values[:-1])
    ax.add_collection(lc)

    ax.autoscale_view()  # ensure axes show everything

    # proxy artist for legend
    return Line2D([0], [0], color=cmap(norm(df_in["time"].mean())), lw=2, label=label)

In [ ]:
t_min = min(df["time"].min(), df2["time"].min())
t_max = max(df["time"].max(), df2["time"].max())

norm = colors.Normalize(vmin=t_min, vmax=t_max)
fig, ax = plt.subplots()
h1 = colored_trajectory(df, ax, label="Jamming", cmap=cmap, norm=norm)
h2 = colored_trajectory(df2, ax, label="No Jamming", cmap=cmap2, norm=norm)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect("equal")

ax.legend(handles=[h1, h2], loc="upper left")
ax.set_aspect("equal")
# First colorbar for df
cax1 = fig.add_axes([0.92, 0.55, 0.02, 0.35])  # [left, bottom, width, height]
sm1 = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
sm1.set_array([])
cbar1 = plt.colorbar(sm1, cax=cax1)
cbar1.set_label("Time")

# Colorbar 2
cax2 = fig.add_axes([0.92, 0.1, 0.02, 0.35])  # stacked below
sm2 = plt.cm.ScalarMappable(norm=norm, cmap=cmap2)
sm2.set_array([])
cbar2 = plt.colorbar(sm2, cax=cax2)
cbar2.set_label("Time")


plt.show()

In [ ]:
h1